In [1]:
import glob
import os
import cv2
import numpy as np
from keras.models import load_model
from skimage import color

from dataset.utils.inception_utils import inception_resnet_v2_predict
from dataset.utils.resize import resize_pad_frame
from dataset.utils.shared import frames_per_video, default_nn_input_width, default_nn_input_height, resnet_input_height, resnet_input_width, dir_test, dir_test_results
from model import FusionLayer
import seaborn as sns 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

sns.set(rc={'figure.figsize':(20,12)})

Using TensorFlow backend.


# Utility functions

In [2]:
def get_video(file):
    '''
    Parameters
    ----------
    file - path to video file 
    
    Returns
    -------
    frames - frames array of the video
    '''
    
    video = cv2.VideoCapture(file)
    frames = []
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        frames.append(frame)
    frames = np.asarray(frames)
    return frames


def get_lab_layer(frames):
    '''
        Parameters
        -----------
        frames - color/gray video frames with 3 chanels
        
        Returns
        -------
        (rgb2lab, gray2lab) - RGB frames converted to LAB, GRAY frames converted to LAB
    '''
    rgb2lab_frames = []
    gray2lab_frames = []

    for frame in frames:
        resized_frame = resize_pad_frame(frame, (default_nn_input_height, default_nn_input_width), equal_padding=True)

        rgb2lab_frame = color.rgb2lab(resized_frame)
        rgb2lab_frames.append(rgb2lab_frame)
        
        rgb2gray_frame = color.rgb2gray(resized_frame)
        
#         Display Grayscale frame
#         cv2.imshow('grey', rgb2gray_frame)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
        
        gray2rgb_frame = color.gray2rgb(rgb2gray_frame)
        lab_frame = color.rgb2lab(gray2rgb_frame)
        gray2lab_frames.append(lab_frame)

    return np.asarray(rgb2lab_frames), np.asarray(gray2lab_frames)


def preprocess_frames(gray2lab_frames):
    '''
    Parameters
    ---------
    gray2lab_frames - LAB frames of Grayscale video
    
    Returns
    -------
    processed_l_layer - L Layer processed (L/50 - 1)
    '''
    processed = np.empty(gray2lab_frames.shape)
    
    processed[:, :, :, 0] = np.divide(gray2lab_frames[:, :, :, 0], 50) - 1  # data loss
    processed[:, :, :, 1] = np.divide(gray2lab_frames[:, :, :, 1], 128)
    processed[:, :, :, 2] = np.divide(gray2lab_frames[:, :, :, 2], 128)
    
    processed_l_layer = processed[:, :, :, np.newaxis, 0]
    
    return processed_l_layer


def get_resnet_records(frames):
    '''
    Parameters
    ----------
    frames - original frames without color conversion or resizing
    
    Details
    -------
    Implementation adopted from Deep Kolorization implementation
    
    Returns
    -------
    predictions - restnet predictions 
    '''
    resnet_input = []
    for frame in frames:
        resized_frame = resize_pad_frame(frame, (resnet_input_height, resnet_input_width))
        gray_scale_frame = cv2.cvtColor(resized_frame, cv2.COLOR_RGB2GRAY)
        gray_scale_frame_colored = cv2.cvtColor(gray_scale_frame, cv2.COLOR_GRAY2RGB)
        resnet_input.append(gray_scale_frame_colored)
    resnet_input = np.asarray(resnet_input)

    predictions = inception_resnet_v2_predict(resnet_input)
    return predictions


def getInputRange(frames_count, time_steps, current_frame):
    '''
    Deciding the moving window
    '''
    # this function should change according to our selection of
    frame_selection = []
    last_selection = current_frame
    for i in range(current_frame, current_frame - time_steps, -1):
        if (i < 0):
            frame_selection.append(last_selection)
        else:
            frame_selection.append(i)
            last_selection = i
    frame_selection = frame_selection[::-1]
    return frame_selection


def get_nn_input(l_layer, resnet_out):
    '''
    Define the flowchroma input
    '''
    frames_count = l_layer.shape[0]
    time_steps = frames_per_video
    X = []
    Y = []

    for i in range(frames_count):
        frame_index_selection = getInputRange(frames_count, time_steps, i)
        frame_selection = []
        resnet_selection = []
        for j in frame_index_selection:
            frame_selection.append(l_layer[j])
            resnet_selection.append(resnet_out[j])
        X.append(frame_selection)
        Y.append(resnet_selection)

    X = np.asarray(X)
    Y = np.asarray(Y)
    return [X, Y]


def post_process_predictions(original_l_layers, predicted_AB_layers):
    '''
    Combine original L layer and predicted AB Layers
    '''
    time_steps = frames_per_video
    total_frames = original_l_layers.shape[0]
    predicted_frames = []
    for i in range(total_frames):
        l_layer = original_l_layers[i]
        a_layer = np.multiply(predicted_AB_layers[i, time_steps - 1, :, :, 0], 128) # select the first frame outof three predictions
        b_layer = np.multiply(predicted_AB_layers[i, time_steps - 1, :, :, 1], 128)
        frame = np.empty((240, 320, 3))
        frame[:, :, 0] = l_layer
        frame[:, :, 1] = a_layer
        frame[:, :, 2] = b_layer
        #frame = color.lab2rgb(frame)
        predicted_frames.append(frame)
    return np.asarray(predicted_frames)


def save_output_video(frames, output_file):
    '''
    Save the output video
    '''
    fps = 20
    size = (320, 240)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_file, fourcc, fps, size)
    count = 0
    for frame in frames:
        final_out = color.lab2rgb(frame)
        #final_out = np.asarray(final_out).astype(int)
        #plt.imshow(final_out)
        final_out_write_video = final_out*255 # color.lab2rgb results values in [0,1]
        final_out_write_video = final_out_write_video.astype(np.uint8)
#         cv2.imshow('image', final_out)
        count+=1
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
        out.write(final_out_write_video)
    out.release()

    # write to output_file

# Pre-processing

In [4]:
frames = get_video("cheetah_black.mp4")
(rgb2lab_frames, gray2lab_frames) = get_lab_layer(frames)
processed_l_layer = preprocess_frames(gray2lab_frames)
print('running resnet model')
predictions = get_resnet_records(frames)
print('Combining L laber and resnet out')
X = get_nn_input(processed_l_layer, predictions)

running resnet model
Combining L laber and resnet out


# Predicting

In [12]:
from model.flowchroma_network import FlowChroma
from keras.layers import Input
from keras.models import load_model
from keras.optimizers import Adam

# run flowchroma model
# ckpts = glob.glob("checkpoints/*.hdf5")
# latest_ckpt = max(ckpts, key=os.path.getctime)
frames_per_video = 3
default_nn_input_height = 240
default_nn_input_width = 320

time_steps, h, w = frames_per_video, default_nn_input_height, default_nn_input_width

enc_input = Input(shape=(time_steps, h, w, 1), name='encoder_input')
incep_out = Input(shape=(time_steps, 1000), name='inception_input')

model = FlowChroma([enc_input, incep_out]).build()
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss='mse', metrics=['accuracy'])
model.summary()
# generate_model_summaries(model)
# print("loading from checkpoint:", latest_ckpt)
# model = load_model(latest_ckpt, custom_objects={'FusionLayer': FusionLayer})
# predictions = []
# for i in range(X[0].shape[0]):
#     predictions.append(model.predict([X[0][i:i+1],X[1][i:i+1]])[0])
# predictions = np.asarray(predictions)
# print("Flowchroma model predictions calculated")

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 3, 240, 320,  0                                            
__________________________________________________________________________________________________
encoder_conv1 (TimeDistributed) (None, 3, 120, 160,  640         encoder_input[0][0]              
__________________________________________________________________________________________________
encoder_conv2 (TimeDistributed) (None, 3, 120, 160,  73856       encoder_conv1[0][0]              
__________________________________________________________________________________________________
encoder_conv3 (TimeDistributed) (None, 3, 60, 80, 12 147584      encoder_conv2[0][0]              
____________________________________________________________________________________________

In [ ]:
# model = load_model(latest_ckpt, custom_objects={'FusionLayer': FusionLayer})
predictions = []
for i in range(X[0].shape[0]):
    predictions.append(model.predict([X[0][i:i+1],X[1][i:i+1]])[0])
predictions = np.asarray(predictions)
print("Flowchroma model predictions calculated")

# Post-processing

In [ ]:
frame_predictions = post_process_predictions(gray2lab_frames[:,:,:,0], predictions)


# Visualization

In [12]:
save_output_video(frame_predictions, dir_test_results+'/2.avi')
# print(np.mean(frame_predictions[1]))

# ---------------------------------------------------------------------------------------------------
# Other stuff

In [ ]:
print(rgb2lab_frames.shape)
print(np.amax(rgb2lab_frames[0,:,:,0]))
sns.heatmap(rgb2lab_frames[0,80:90,80:90,0],annot=True )


In [ ]:
print(gray2lab_frames.shape)
print(np.amax(gray2lab_frames[0,:,:,0]))
sns.heatmap(gray2lab_frames[0,80:90,80:90,0],annot=True )


In [ ]:
print(processed_l_layer.shape)
print(np.amax(processed_l_layer[0,:,:,0]))
sns.heatmap(processed_l_layer[0,80:90,80:90,0],annot=True )


# L Layer Comparison

In [ ]:
fig, (ax,ax2) = plt.subplots(ncols=2)
fig.subplots_adjust(wspace=0.01)

sns.heatmap(frame_predictions[0,80:90,80:90,0],annot=True, ax=ax )
sns.heatmap(rgb2lab_frames[0,80:90,80:90,0],annot=True, ax=ax2 )


# A Layer Comparison

In [ ]:
fig, (ax,ax2) = plt.subplots(ncols=2)
fig.subplots_adjust(wspace=0.01)
sns.heatmap(frame_predictions[1,80:90,80:90,1],annot=True, ax=ax )
sns.heatmap(rgb2lab_frames[1,80:90,80:90,1],annot=True, ax=ax2 )

# B Layer comparison

In [ ]:
fig, (ax,ax2) = plt.subplots(ncols=2)
fig.subplots_adjust(wspace=0.01)
sns.heatmap(frame_predictions[0,80:90,80:90,2],annot=True, ax=ax )
sns.heatmap(rgb2lab_frames[0,80:90,80:90,2],annot=True, ax=ax2 )